In [74]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import p
import meshplot as mp

In [75]:
def all_combos(choices):
    print("choices", choices)
    if not choices:
        yield []
        return

    for left_choice in choices[0]:
        print("left choice", print(left_choice))
        for right_choices in all_combos(choices[1:]):
            print("right choice", right_choices)
            yield [left_choice] + right_choices


In [76]:
class Node:
    """
    Class Node
    """
    def __init__(self, value, radius, left = None, right = None, position = None, cl_prob= None, combos = None):
        self.left = left
        self.data = value
        self.radius = radius
        self.position = position
        self.right = right
        self.prob = cl_prob
        self.combos = combos
        self.children = [self.left, self.right]

    def __str__(self):
        """
        Display the node's value, and then its children in brackets if it has any.
        """
        if self.children:
            return "%s %s" % (self.data, self.children)
        else:
            return str(self.data)

    def __repr__(self):
        return str(self)

    def all_subtrees(self, max_depth):
    
        yield Node(self.data, self.radius)

        if max_depth > 0:
            # For each child, get all of its possible sub-trees.
            self.child()
            print("children", self.children)
            #child_subtrees = [list(self.children[i].all_subtrees(max_depth - 1)) for i in range(len(self.children)) if self.children[i] is not None]
            child_subtrees = [list(self.children[i].all_subtrees(max_depth - 1)) for i in range(len(self.children))]
            print("child subtrees", child_subtrees)
            # Now for the n children iterate through the 2^n possibilities where
            # each child's subtree is independently present or not present. The
            # i-th child is present if the i-th bit in "bits" is a 1.
            for bits in range(1, 2 ** len(self.children)):
                #for combos in all_combos([child_subtrees[i] for i in range(len(self.children)) if bits & (1 << i) != 0 and self.children[i] is not None]):
                for combos in all_combos([child_subtrees[i] for i in range(len(self.children)) if bits & (1 << i) != 0 ]):
                    yield Node(self.data, self.radius, combos = combos)

    def child(self):
        self.children = [self.left, self.right]
    def toGraph( self, graph, index, dec, proc=True):
        
        
        radius = self.radius#.cpu().detach().numpy()
        print("radius", radius)
        if dec:
            radius= radius[0]
        #print("posicion", self.data, radius)
        #print("right", self.right)
        
        #graph.add_nodes_from( [ (index, {'posicion': radius[0:3], 'radio': radius[3] } ) ])
        graph.add_nodes_from( [ (self.data, {'posicion': radius[0:3], 'radio': radius[3] } ) ])
        

        if self.right is not None:
            #leftIndex = self.right.toGraph( graph, index + 1, dec)#
            self.right.toGraph( graph, index + 1, dec)#
            
            #graph.add_edge( index, index + 1 )
            graph.add_edge( self.data, self.right.data )
            #if proc:
            #    nx.set_edge_attributes( graph, {(index, index+1) : {'procesada':False}})
        
            if self.left is not None:
                #retIndex = self.left.toGraph( graph, leftIndex, dec )#
                self.left.toGraph( graph, 0, dec )#

                #graph.add_edge( index, leftIndex)
                graph.add_edge( self.data, self.left.data)
                #if proc:
                #    nx.set_edge_attributes( graph, {(index, leftIndex) : {'procesada':False}})
            
            else:
                #return leftIndex
                return

        else:
            #return index + 1
            return

In [77]:
def createNode(data, radius, position = None, left = None, right = None, cl_prob = None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, position, left, right, cl_prob)

def height(root):
    # Check if the binary tree is empty
    if root is None:
        return 0 
    # Recursively call height of each node
    leftAns = height(root.left)
    rightAns = height(root.right)
    
    # Return max(leftHeight, rightHeight) at each iteration
    return max(leftAns, rightAns) + 1

def printCurrentLevel(root, level):
    if root is None:
        return
    if level == 1:
        print(root.data, root.radius, end=" ")
    elif level > 1:
        printCurrentLevel(root.left, level-1)
        printCurrentLevel(root.right, level-1)

def printLevelOrder(root):
    h = height(root)
    for i in range(1, h+1):
       printCurrentLevel(root, i)

def serialize(root):
        
    def post_order(root):
        if root:
            post_order(root.left)
            post_order(root.right)
            ret[0] += str(root.data)+'_'+ str(root.radius) +';'
                
        else:
            ret[0] += '#;'           

    ret = ['']
    post_order(root)

    return ret[0][:-1]  # remove last 

def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')

    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)

    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'])

In [78]:

root = Node(1, [1., 1., 1., 1.])#0
root.left = createNode(3, [3., 3., 3., 3.]) #0.4
root.left.right = createNode(6, [6., 6., 6., 6.])#1 

root.left.left = createNode(7, [4., 4., 4., 4.]) 
root.right = createNode(2, [10., 2., 2., 2.]) #0.2
root.right.right = createNode(4, [4., 4., 4., 4.]) #0.6

root.right.left = createNode(5, [5., 5., 5., 5.]) #0.8


tree = root

for subtree in tree.all_subtrees(2):
    print("///")
    print ("subtree", subtree)
    #plotTree(subtree, False)
    #print(subtree.right)
    #print(subtree.left)

///
subtree 1 [None, None]
children [3 [None, None], 2 [None, None]]
children [7 [None, None], 6 [None, None]]
child subtrees [[7 [None, None]], [6 [None, None]]]
choices [[7 [None, None]]]
7 [None, None]
left choice None
choices []
right choice []
choices [[6 [None, None]]]
6 [None, None]
left choice None
choices []
right choice []
choices [[7 [None, None]], [6 [None, None]]]
7 [None, None]
left choice None
choices [[6 [None, None]]]
6 [None, None]
left choice None
choices []
right choice []
right choice [6 [None, None]]
children [5 [None, None], 4 [None, None]]
child subtrees [[5 [None, None]], [4 [None, None]]]
choices [[5 [None, None]]]
5 [None, None]
left choice None
choices []
right choice []
choices [[4 [None, None]]]
4 [None, None]
left choice None
choices []
right choice []
choices [[5 [None, None]], [4 [None, None]]]
5 [None, None]
left choice None
choices [[4 [None, None]]]
4 [None, None]
left choice None
choices []
right choice []
right choice [4 [None, None]]
child subtree